In [1]:
!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
autograd is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=e363270abd72d1ac032c2a1bcc1b593fa522c72323c438cd23f2ed81656f2298
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl


In [2]:
import pandas as pd
import numpy as np

import optuna
import xgboost
import lightgbm
import catboost
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostRegressor, CatBoostClassifier

from metric import score
from scipy.stats import rankdata, kendalltau
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings
import gc
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
test = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')

In [4]:
from lifelines import KaplanMeierFitter, CoxPHFitter, NelsonAalenFitter

def kmfitter(data, time_col='efs_time', event_col='efs'):
    fitter = KaplanMeierFitter()
    fitter.fit(data[time_col], data[event_col])
    new_target = fitter.survival_function_at_times(data[time_col]).values
    return new_target

def nafitter(data, time_col='efs_time', event_col='efs'):
    data=data.copy()
    naf = NelsonAalenFitter(nelson_aalen_smoothing=0)
    naf.fit(durations=data[time_col], event_observed=data[event_col])
    return naf.cumulative_hazard_at_times(data[time_col]).values*-1

In [5]:
def create_columns(df):
   # Create HLA sum columns
    df['hla_nmdp_6'] = df['hla_match_a_low'].fillna(0) + df['hla_match_b_low'].fillna(0) + df['hla_match_drb1_high'].fillna(0)
   
    df['hla_low_res_6'] = df['hla_match_a_low'].fillna(0) + df['hla_match_b_low'].fillna(0) + df['hla_match_drb1_low'].fillna(0)
   
    df['hla_high_res_6'] = df['hla_match_a_high'].fillna(0) + df['hla_match_b_high'].fillna(0) + df['hla_match_drb1_high'].fillna(0)
   
    df['hla_low_res_8'] = df['hla_match_a_low'].fillna(0) + df['hla_match_b_low'].fillna(0) + df['hla_match_c_low'].fillna(0) + df['hla_match_drb1_low'].fillna(0)
   
    df['hla_high_res_8'] = df['hla_match_a_high'].fillna(0) + df['hla_match_b_high'].fillna(0) + df['hla_match_c_high'].fillna(0) + df['hla_match_drb1_high'].fillna(0)
   
    df['hla_low_res_10'] = df['hla_match_a_low'].fillna(0) + df['hla_match_b_low'].fillna(0) + df['hla_match_c_low'].fillna(0) + df['hla_match_drb1_low'].fillna(0) + df['hla_match_dqb1_low'].fillna(0)
   
    df['hla_high_res_10'] = df['hla_match_a_high'].fillna(0) + df['hla_match_b_high'].fillna(0) + df['hla_match_c_high'].fillna(0) + df['hla_match_drb1_high'].fillna(0) + df['hla_match_dqb1_high'].fillna(0)
    
    df['donor_age-age_at_hct']=df['donor_age']-df['age_at_hct']

    df['year_hct'] = df['year_hct'] - 2000
    
    df['comorbidity_score+karnofsky_score']=df['comorbidity_score']+df['karnofsky_score']
    
    df['comorbidity_score-karnofsky_score']=df['comorbidity_score']-df['karnofsky_score']
    
    df['comorbidity_score*karnofsky_score']=df['comorbidity_score']*df['karnofsky_score']
    
    df['comorbidity_score/karnofsky_score']=df['comorbidity_score']/df['karnofsky_score']

    return df

train = create_columns(train)
test = create_columns(test)

In [6]:
class TargetEncoder:
    def __init__(self, cat_feats, target='mean_km_dras', kfold=5, smooth=20, agg="mean", 
                 target_cols=['efs', 'efs_time', 'km', 'dras', 'mean_km_dras']):
        """
        Initialize the target encoder.
        
        Parameters:
        -----------
        cat_feats : list
            List of categorical feature names to encode
        target : str
            Target variable name
        kfold : int
            Number of folds for cross-validation
        smooth : int
            Smoothing parameter
        agg : str
            Aggregation method ("mean", "median", "min", "max", "nunique")
        target_cols : list
            List of target columns to move to the right
        """
        self.cat_feats = cat_feats
        self.target = target
        self.kfold = kfold
        self.smooth = smooth
        self.agg = agg
        self.encoding_dict = {feat: {} for feat in cat_feats}
        self.global_value = None
        self.target_cols = target_cols
        
    def _calculate_smooth_target_encoding(self, df, col, target):
        """Calculate target encoding for a single feature"""
        agg_df = df[[col, target]].groupby(col)[target].agg([self.agg, 'count']).reset_index()
        
        if self.agg == "nunique":
            agg_df['TE_tmp'] = agg_df[self.agg] / agg_df['count']
        else:
            agg_df['TE_tmp'] = ((agg_df[self.agg] * agg_df['count']) + 
                               (self.global_value * self.smooth)) / (agg_df['count'] + self.smooth)
        return agg_df
    
    def _get_global_value(self, df):
        """Calculate global statistics for the target variable"""
        if self.agg == "mean":
            return df[self.target].mean()
        elif self.agg == "median":
            return df[self.target].median()
        elif self.agg == "min":
            return df[self.target].min()
        elif self.agg == "max":
            return df[self.target].max()
        elif self.agg == "nunique":
            return 0
        return None
    
    def _reorder_columns(self, df):
        """
        Reorder columns to move target columns to the right
        """
        # Get all columns except target columns
        non_target_cols = [col for col in df.columns if col not in self.target_cols]
        
        # Get existing target columns (some might not be present)
        existing_target_cols = [col for col in self.target_cols if col in df.columns]
        
        # Return reordered dataframe
        return df[non_target_cols + existing_target_cols]
    
    def fit(self, train):
        if not isinstance(self.cat_feats, list) or len(self.cat_feats) == 0:
            raise ValueError("cat_feats must be a non-empty list of column names")
        if self.target not in train.columns:
            raise ValueError(f"Target column '{self.target}' not found in training data")
    
        kf = KFold(n_splits=self.kfold, shuffle=True, random_state=42)  # Use proper CV splitting
    
        self.global_value = self._get_global_value(train)
    
        train = train.copy()
        train['fold'] = -1  # Initialize fold column
    
        for fold, (train_idx, valid_idx) in enumerate(kf.split(train)):
            train.loc[valid_idx, 'fold'] = fold  # Assign fold index
    
        for feat in self.cat_feats:
            self.encoding_dict[feat] = {}
    
            for fold in range(self.kfold):
                df_train = train[train['fold'] != fold]  # Exclude fold
                agg_df = self._calculate_smooth_target_encoding(df_train, feat, self.target)
                self.encoding_dict[feat][fold] = agg_df  # Store fold-wise encoding
    
            # Global encoding for test set
            self.encoding_dict[feat]['final'] = self._calculate_smooth_target_encoding(train, feat, self.target)
    
        train.drop(columns=['fold'], inplace=True)  # Remove fold column
        return self
    
    def transform(self, df, is_train=False, fold_idx=None):
        """
        Apply encoding to a dataset.
        
        Parameters:
        -----------
        df : pandas.DataFrame
            Dataset to transform
        is_train : bool
            Whether this is training data
        fold_idx : int, optional
            Fold index for training data
            
        Returns:
        --------
        pandas.DataFrame
            Transformed dataset with target columns on the right
        """
        df = df.copy()
        
        if is_train:
            # Add kfold column for training data
            df['kfold'] = df.index % self.kfold
        
        # Process each categorical feature separately
        for feat in self.cat_feats:
            te_col = f'TE_{self.agg.upper()}_{feat}'
            
            if is_train and fold_idx is not None:
                # Use appropriate fold encoding
                agg_df = self.encoding_dict[feat][fold_idx]
            else:
                # Use final encoding for validation/test
                agg_df = self.encoding_dict[feat]['final']
                
            # Apply encoding
            df_tmp_m = df[[feat]].merge(agg_df, how='left', on=feat)
            df[te_col] = df_tmp_m['TE_tmp'].fillna(self.global_value).astype("float32")
        
        # Remove the kfold column if it was added
        if 'kfold' in df.columns:
            df.drop('kfold', axis=1, inplace=True)
        
        # Reorder columns to move target columns to the right
        df = self._reorder_columns(df)
        
        return df

In [7]:
rmv = ['ID', 'efs', 'efs_time', 'efs_time2', 'km','mean_km_dras', 'dras', 'na']
feats = [col for col in train.columns if col not in rmv]

cat_feats = []
#num_feats = []
for c in feats:
    if train[c].dtype == 'object':
        cat_feats.append(c)
        train[c] = train[c].fillna('Unknown')
        test[c] = test[c].fillna('Unknown')
    else:
        #num_feats.append(c)
        train[c] = train[c].fillna(-1)
        test[c] = test[c].fillna(-1)

covariate = ['year_hct', 'age_at_hct', 'donor_age', 'comorbidity_score','karnofsky_score','dri_score']

In [8]:
combined = pd.concat([train, test])
for col in feats:
    ftype = "numerical"
    if col in cat_feats:
        combined[col], _ = combined[col].factorize()
        combined[col] -= combined[col].min()
        combined[col].astype('int32')
        combined[col].astype('category')
        ftype = "Categorical"
    else:
        if combined[col].dtype == 'int64':
            combined[col].astype('int32')
        if combined[col].dtype == 'float64':
            combined[col].astype('float32')

train = combined.iloc[:len(train)].copy()
test = combined.iloc[len(train):].reset_index(drop=True).copy()

In [9]:
def time_weight(t, alpha):
    """Calculate time-based weight using exponential decay"""
    return np.exp(-alpha * t)

def hazard(t, cols, beta, h0):
    """Calculate hazard rate with multiple features"""
    cols = np.array(cols)
    linear_pred = np.sum(beta * cols)
    return h0 * np.exp(linear_pred)

def dras_survival(efs_time, efs, data, cols, alpha, beta, h0):
    """Calculate individual patient survival probability
#    
#    Args:
#        efs_time (float): Individual patient's event/censoring time in months
#        efs (int): Event indicator (0=event, 1=censored)
#        data (pd.DataFrame): Input data
#        cols (list): List of covariate values
#        alpha (float): Time weight parameter
#        beta (array-like): Array of coefficients matching cols length
#        h0 (float): Baseline hazard
#    """
    survival_prob = 1.0
    
    # Get the individual patient's time
    patient_time = float(efs_time.iloc[0]) if hasattr(efs_time, 'iloc') else float(efs_time)
    
    # Convert months to integer number of time points
    n_timepoints = int(np.ceil(patient_time))
    
    # Calculate survival probability up to the patient's time
    for t in range(1, n_timepoints + 1):
        weighted_hazard = time_weight(t, alpha) * hazard(t, cols, beta, h0)    
        survival_prob *= np.exp(-weighted_hazard)
    # Get scalar value for efs
    efs_value = float(efs.iloc[0]) if hasattr(efs, 'iloc') else float(efs)
    
    # If event occurred (efs=0), apply additional penalty based on actual time
    if efs_value == 0:
        hazard_at_event = hazard(patient_time, cols, beta, h0)
        penalty = np.exp(- hazard_at_event * np.exp(-alpha * patient_time))
        survival_prob *= penalty
    
    return survival_prob

In [10]:
alpha = 1e-5
beta = 1e-5
h0 = 0.12

train['dras'] = train.apply(lambda row: dras_survival(
    efs_time=row['efs_time'],
    efs=row['efs'], 
    data=train,
    cols=[row[col] for col in covariate],
    alpha=alpha,
    beta=beta,
    h0=h0
), axis=1).astype('float32')

train['km'] = kmfitter(train)
train['na'] = nafitter(train)

In [11]:
train['mean_km_dras'] = train.apply(
    lambda row: (row['km'] + row['dras']) / 1.5, axis=1
)
train.loc[train.efs == 0, "na"] = (-(-train.loc[train.efs == 0, "na"])**0.5)

In [12]:
def convert_to_categorical(df, categorical_columns):
    """Convert specified columns to categorical dtype"""
    df = df.copy()
    for col in categorical_columns:
        if col in df.columns:
            df[col] = df[col].astype('category')
    return df

def evaluate_metrics(y_true, y_pred):
    """Calculate multiple evaluation metrics"""
    return {
        'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
        'mae': mean_absolute_error(y_true, y_pred),
        'r2': r2_score(y_true, y_pred)
    }

In [13]:
VALIDATE = True
xgb_params_km = {
    'n_estimators': 3000,
    "max_depth": 12,
    "learning_rate": 0.009112211792482224,
    "min_child_weight": 10,
    "subsample": 0.8289720172641504,
    "colsample_bytree": 0.5214350470812086,
    "colsample_bylevel": 0.5210943101555645,
    "colsample_bynode": 0.6426572333018175,
    "gamma": 1.1003991927886534e-08,
    "reg_alpha": 1.4525103196306596e-08,
    "reg_lambda": 0.04424304066465823,
    "max_leaves": 40,
    "max_bin": 279,
    "grow_policy": "lossguide",
    "sample_type": "weighted",
    "normalize_type": "tree",
    "rate_drop": 0.47281245040192416,
    "skip_drop": 0.35445024142787784,
    'enable_categorical': True,
    'tree_method': 'hist',
    'grow_policy': 'depthwise',
}

xgb_params_na = {
    'n_estimators': 3000,
    "max_depth": 5,
    "learning_rate": 0.011581087169803534,
    "min_child_weight": 5,
    "subsample": 0.881780143914574,
    "colsample_bytree": 0.5120164480652475,
    "colsample_bylevel": 0.6017719529609797,
    "colsample_bynode": 0.5799467228083436,
    "gamma": 0.004791567740160683,
    "reg_alpha": 1.025197779142139,
    "reg_lambda": 6.35043697821767e-08,
    "max_leaves": 52,
    "max_bin": 205,
    "grow_policy": "lossguide",
    "sample_type": "uniform",
    "normalize_type": "tree",
    "rate_drop": 0.49785180560961273,
    "skip_drop": 0.36256480631225946,
    'enable_categorical': True,
    'tree_method': 'hist',
    'grow_policy': 'depthwise',
}

lgb_params_km = {
    'n_estimators': 1997,
    'learning_rate': 0.02536409136667196,
    'max_depth': 5,
    'num_leaves': 163,
    'min_child_samples': 55,
    'max_bin': 449,
    'min_data_in_bin': 92,
    'subsample': 0.6078933370188859,
    'subsample_freq': 6,
    'bagging_fraction': 0.9338265496147928,
    'bagging_freq': 1,
    'colsample_bytree': 0.8901972542165217,
    'colsample_bynode': 0.6283997865098214,
    'feature_fraction': 0.8216565096391157,
    'reg_alpha': 0.0015849033946108194,
    'reg_lambda': 0.0004341663911716948,
    'min_split_gain': 9.874677312705849e-06,
    'min_gain_to_split': 0.005579298307975093,
    'min_child_weight': 0.010236465301988055,
    'min_sum_hessian_in_leaf': 6.667673208938888,
    'path_smooth': 4,
    'max_cat_threshold': 23,
    'cat_smooth': 24.27640728987526,
    'cat_l2': 33.08936852294363,
    'extra_trees': True,
    'boost_from_average': True,
    'is_unbalance': False,
    'drop_rate': 0.23958204122015014,
    'max_drop': 58,
    'skip_drop': 0.40847077905040124,
    'max_depth_policy': 'lossguide',
    'top_rate': 0.8216571675941808,
    'other_rate': 0.12377038643622994,
    'min_data_per_group': 92,
    'max_cat_to_onehot': 10,
    'n_jobs': -1,
    'verbose': -1,
    'force_col_wise': True,
    'early_stopping_round': 500,
    'first_metric_only': True,
    'feature_fraction_seed': 42,
    'bagging_seed': 42,
    'drop_seed': 42,
    'data_random_seed': 42,
    'metric': 'rmse',
    'objective': 'regression',
}

lgb_params_na = {
    'n_estimators': 1459,
    'learning_rate': 0.02412424877593639,
    'max_depth': 10,
    'num_leaves': 167,
    'min_child_samples': 8,
    'max_bin': 448,
    'min_data_in_bin': 63,
    'subsample': 0.7128249545082955,
    'subsample_freq': 2,
    'bagging_fraction': 0.7583182101510887,
    'bagging_freq': 1,
    'colsample_bytree': 0.5992946113172252,
    'colsample_bynode': 0.6564440734731318,
    'feature_fraction': 0.6456121257238647,
    'reg_alpha': 0.7028251350392851,
    'reg_lambda': 1.846427873083449,
    'min_split_gain': 1.1774126319233929e-06,
    'min_gain_to_split': 0.0051669663271333675,
    'min_child_weight': 0.0015626385043121316,
    'min_sum_hessian_in_leaf': 0.3213377445627896,
    'path_smooth': 6,
    'max_cat_threshold': 34,
    'cat_smooth': 14.640109654767594,
    'cat_l2': 10.160582046890028,
    'extra_trees': True,
    'boost_from_average': False,
    'is_unbalance': True,
    'drop_rate': 0.21586393703061418,
    'max_drop': 62,
    'skip_drop': 0.471887756972435,
    'max_depth_policy': 'lossguide',
    'top_rate': 0.9025231413059317,
    'other_rate': 0.18467093692504075,
    'min_data_per_group': 200,
    'max_cat_to_onehot': 7,
    'n_jobs': -1,
    'verbose': -1,
    'force_col_wise': True,
    'early_stopping_round': 500,
    'first_metric_only': True,
    'feature_fraction_seed': 42,
    'bagging_seed': 42,
    'drop_seed': 42,
    'data_random_seed': 42,
    'metric': 'rmse',
    'objective': 'regression',
}

lgb2_params_km = {
    "n_estimators": 1822,
    "learning_rate": 0.011913333725509567,
    "max_depth": 6,
    "num_leaves": 211,
    "min_child_samples": 55,
    "max_bin": 236,
    "min_data_in_bin": 16,
    "subsample": 0.672493728193822,
    "subsample_freq": 1,
    "bagging_fraction": 0.7714272498918423,
    "bagging_freq": 4,
    "colsample_bytree": 0.816324075352981,
    "colsample_bynode": 0.7005570052563428,
    "feature_fraction": 0.5163679100795492,
    "reg_alpha": 0.10871673070184677,
    "reg_lambda": 1.0144925624679776e-06,
    "min_split_gain": 5.8493221205312494e-05,
    "min_gain_to_split": 0.006231346860399903,
    "min_child_weight": 0.029362848048560056,
    "min_sum_hessian_in_leaf": 0.018031327096960915,
    "path_smooth": 5,
    "max_cat_threshold": 57,
    "cat_smooth": 42.543973513033976,
    "cat_l2": 12.813963218673436,
    "extra_trees": False,
    "boost_from_average": True,
    "is_unbalance": True,
    "drop_rate": 0.4767162828494658,
    "max_drop": 60,
    "skip_drop": 0.22138323515275962,
    "max_depth_policy": "lossguide",
    "top_rate": 0.7089420922026488,
    "other_rate": 0.26831606996993906,
    "min_data_per_group": 132,
    "max_cat_to_onehot": 5,
    "feature_fraction_bynode": 0.6324579559727216,
    'is_provide_training_metric': False,
    'histogram_pool_size': -1,
    'n_jobs': -1,
    'verbose': -1,
    'force_col_wise': True,
    'early_stopping_round': 500,
    'first_metric_only': True,
    'feature_fraction_seed': 42,
    'bagging_seed': 42,
    'drop_seed': 42,
    'data_random_seed': 42,
    'metric': 'rmse',
    'objective': 'regression',
}

lgb2_params_na = {
    "n_estimators": 1968,
    "learning_rate": 0.04789544258839567,
    "max_depth": 3,
    "num_leaves": 205,
    "min_child_samples": 48,
    "max_bin": 308,
    "min_data_in_bin": 51,
    "subsample": 0.6457237012645485,
    "subsample_freq": 1,
    "bagging_fraction": 0.6354529485719242,
    "bagging_freq": 9,
    "colsample_bytree": 0.8515546800822218,
    "colsample_bynode": 0.6379308511725845,
    "feature_fraction": 0.8754220686147757,
    "reg_alpha": 0.07412641215784554,
    "reg_lambda": 1.2395279882892064e-07,
    "min_split_gain": 0.013224235963676848,
    "min_gain_to_split": 0.003907336752292077,
    "min_child_weight": 1.7676533406396693,
    "min_sum_hessian_in_leaf": 0.02484128472285958,
    "path_smooth": 8,
    "max_cat_threshold": 26,
    "cat_smooth": 16.260610011793688,
    "cat_l2": 33.646925619146806,
    "extra_trees": True,
    "boost_from_average": True,
    "is_unbalance": False,
    "drop_rate": 0.446024976424448,
    "max_drop": 66,
    "skip_drop": 0.3171847006232032,
    "max_depth_policy": "lossguide",
    "top_rate": 0.7040016739672601,
    "other_rate": 0.10739700360534395,
    "min_data_per_group": 175,
    "max_cat_to_onehot": 7,
    "feature_fraction_bynode": 0.7492901678038486,
    'n_jobs': -1,
    'verbose': -1,
    'force_col_wise': True,
    'early_stopping_round': 500,
    'first_metric_only': True,
    'feature_fraction_seed': 42,
    'bagging_seed': 42,
    'drop_seed': 42,
    'data_random_seed': 42,
    'metric': 'rmse',
    'objective': 'regression',
}
cat_params_km = {
    'iterations': 5000,
    'loss_function': 'RMSE',
    'eval_metric': 'RMSE',
    'random_seed': 42,
    'learning_rate': 0.028800995641969314,
    'min_data_in_leaf': 10,
    'grow_policy': 'Depthwise',
    'bootstrap_type': 'MVS',
    'subsample': 0.6124592065569634,
    'sampling_frequency': 'PerTreeLevel',
    'colsample_bylevel': 0.3262006013682037,
    'l2_leaf_reg': 8.756522522645781e-07,
    'random_strength': 0.0002487262384629418,
    'leaf_estimation_method': 'Newton',
    'one_hot_max_size': 23,
    'max_ctr_complexity': 2,
    'depth': 4,
    'cat_features': cat_feats,
    'verbose': 500
}


cat_params_na = {
    'depth': 7,
    'learning_rate': 0.015,
    'l2_leaf_reg': 4.538529416119618,
    'colsample_bylevel': 0.7,
    'min_data_in_leaf': 10,
    'grow_policy': 'Depthwise',
    'bootstrap_type': 'Bayesian',
    'iterations': 2341,
    'leaf_estimation_method': 'Gradient',
    'sampling_frequency': 'PerTreeLevel',
    'early_stopping_rounds': 500,
    'max_ctr_complexity' : 3,
    'one_hot_max_size': 15,
    'cat_features': cat_feats,
    'task_type': 'CPU',
    'devices': -1,
    'verbose': 500
}

In [14]:
class CrossVal:
    def __init__(self, model):
        self.model = model

    def train_survival_model(self, train, test, cat_feats, covariate, params, target_col='mean_km_dras', folds=5):
        """
        Train survival model with multiple target encoders and cross-validation
        
        Parameters:
        -----------
        train : pd.DataFrame
            Training data
        test : pd.DataFrame
            Test data
        cat_feats : list
            List of categorical features to encode
        covariate : list
            List of covariate features
        params : dict
            Model parameters
        target_col : str
            Name of target column
        folds : int
            Number of cross-validation folds
        """
        # Initialize encoders
        encoders = {
#            'mean': TargetEncoder(cat_feats=cat_feats, target=target_col, agg="mean"),
            'median': TargetEncoder(cat_feats=cat_feats, target=target_col, agg="median"),
#            'max': TargetEncoder(cat_feats=cat_feats, target=target_col, agg="max"),
#            'min': TargetEncoder(cat_feats=cat_feats, target=target_col, agg="min")
        }
    
        # Create stratification groups
        group_cols = train.race_group.astype(str) + train.efs.astype(str)
        
        # Initialize cross-validation
        skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=2025)
        gkf = GroupKFold(n_splits=folds)
        
        # Initialize prediction arrays
        oof_predictions = np.zeros(len(train))
        test_predictions = np.zeros(len(test))
        
        # Define columns to remove
        rmv = ['mean_km_dras','mean_na_dras', 'efs_time2', 'efs', 'efs_time', 'km', 'ID', 'dras', 'na']
        
        # Get all features excluding targets
        all_features = [col for col in train.columns if col not in rmv]
        
        # Identify non-categorical features
        non_cat_features = [col for col in all_features if col not in cat_feats]
        
        # Training loop
        for fold, (tr_idx, val_idx) in enumerate(skf.split(train, group_cols)):
            print(f"\nFold {fold + 1}")
            
            # Split data
            X_tr = train.loc[tr_idx].copy().reset_index(drop=True)
            X_val = train.loc[val_idx].copy().reset_index(drop=True)
            X_test = test.copy()
            
            # Convert categorical columns
            X_tr = convert_to_categorical(X_tr, cat_feats)
            X_val = convert_to_categorical(X_val, cat_feats)
            X_test = convert_to_categorical(X_test, cat_feats)
            
            # Initialize dataframes with non-categorical features
            X_train_encoded = X_tr[non_cat_features].copy()
            X_valid_encoded = X_val[non_cat_features].copy()
            X_test_encoded = X_test[non_cat_features].copy()
            
            # Convert numeric columns to int if possible
            try:
                X_train_encoded = X_train_encoded.astype(int)
                X_valid_encoded = X_valid_encoded.astype(int)
                X_test_encoded = X_test_encoded.astype(int)
            except (ValueError, TypeError):
                pass
            
            # Add categorical columns
            for col in cat_feats:
                if col in X_tr.columns:
                    X_train_encoded[col] = X_tr[col]
                    X_valid_encoded[col] = X_val[col]
                    X_test_encoded[col] = X_test[col]
            
            # Keep track of processed columns to avoid duplicates
            processed_cols = set()
            
            # Apply each encoder
            for name, encoder in encoders.items():
                encoder.fit(X_tr)
                temp_train = encoder.transform(X_tr, is_train=True, fold_idx=fold)
                temp_valid = encoder.transform(X_val, is_train=True, fold_idx=fold)
                temp_test = encoder.transform(X_test, is_train=False)
                
                encoded_cols = [col for col in temp_train.columns 
                              if col.startswith('TE_') and col not in processed_cols]
                processed_cols.update(encoded_cols)
                
                for col in encoded_cols:
                    X_train_encoded[col] = temp_train[col]
                    X_valid_encoded[col] = temp_valid[col]
                    X_test_encoded[col] = temp_test[col]
                    
                del temp_train, temp_valid, temp_test, encoded_cols
            
            feature_cols = non_cat_features + list(cat_feats) + list(processed_cols)

            # Print feature information
            print(f"Number of features: {len(feature_cols)}")
            print(f"Non-categorical features: {len(non_cat_features)}")
            print(f"Categorical features: {len(cat_feats)}")
            print(f"Encoded features: {len(processed_cols)}")
            
            if self.model == 'xgb':
                y_train = X_tr[target_col]
                y_valid = X_val[target_col]
                model = XGBRegressor(**params)
                model.fit(
                    X_train_encoded[feature_cols],
                    y_train,
                    eval_set=[(X_valid_encoded[feature_cols], y_valid)],
                    early_stopping_rounds=500,
                    verbose=500
                )

            elif self.model == 'lgb':
                y_train = X_tr[target_col]
                y_valid = X_val[target_col]
                model = LGBMRegressor(**params)
                model.fit(
                    X_train_encoded[feature_cols], 
                    y_train,
                    eval_set=[(X_valid_encoded[feature_cols], y_valid)],
                )
            
            elif self.model == 'lgb2':
                y_train = X_tr[target_col]
                y_valid = X_val[target_col]
                model = LGBMRegressor(**params)
                model.fit(
                    X_train_encoded[feature_cols], 
                    y_train,
                    eval_set=[(X_valid_encoded[feature_cols], y_valid)],
                )
                
            elif self.model == 'cat':
                y_train = X_tr[target_col]
                y_valid = X_val[target_col]
                model = CatBoostRegressor(**params)
                model.fit(
                    X_train_encoded[feature_cols], 
                    y_train,
                    eval_set=[(X_valid_encoded[feature_cols], y_valid)],
                )
                
            elif self.model == 'cat2':
                y_train = X_tr[target_col]
                y_valid = X_val[target_col]
                model = CatBoostRegressor(**params)
                model.fit(
                    X_train_encoded[feature_cols], 
                    y_train,
                    eval_set=[(X_valid_encoded[feature_cols], y_valid)],
                )
            else:
                raise Exception(f"{self.model} is not supported")
            
            # Make predictions
            oof_predictions[val_idx] = model.predict(X_valid_encoded[feature_cols])
            test_predictions += model.predict(X_test_encoded[feature_cols])

            del model, X_train_encoded, X_test_encoded, X_valid_encoded, y_train, y_valid
            del X_tr, X_val, X_test, feature_cols
            gc.collect()
        
        # Average test predictions
        test_predictions /= folds
        
        return oof_predictions, test_predictions

In [15]:
if VALIDATE:
    xgb_km = CrossVal(model='xgb')
    xgb_km_oof_pred, xgb_km_test_pred= xgb_km.train_survival_model(
        train=train,
        test=test,
        cat_feats=cat_feats,
        covariate=covariate,
        target_col='mean_km_dras',
        params=xgb_params_km
    )


Fold 1
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35
[0]	validation_0-rmse:0.27317
[500]	validation_0-rmse:0.24799
[1000]	validation_0-rmse:0.24556
[1500]	validation_0-rmse:0.24464
[2000]	validation_0-rmse:0.24428
[2500]	validation_0-rmse:0.24419
[2999]	validation_0-rmse:0.24410

Fold 2
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35
[0]	validation_0-rmse:0.27301
[500]	validation_0-rmse:0.24793
[1000]	validation_0-rmse:0.24551
[1500]	validation_0-rmse:0.24469
[2000]	validation_0-rmse:0.24432
[2500]	validation_0-rmse:0.24425
[2849]	validation_0-rmse:0.24428

Fold 3
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35
[0]	validation_0-rmse:0.27378
[500]	validation_0-rmse:0.24662
[1000]	validation_0-rmse:0.24369
[1500]	validation_0-rmse:0.24273
[2000]	validation_0-rmse:0.24241
[2500]	validation_0-rmse:0.24244
[2737]	validation_0-rmse:0.24252



In [16]:
if VALIDATE:
    xgb_na = CrossVal(model='xgb')
    xgb_na_oof_pred, xgb_na_test_pred = xgb_na.train_survival_model(
        train=train,
        test=test,
        cat_feats=cat_feats,
        covariate=covariate,
        target_col='na',
        params=xgb_params_na
    )


Fold 1
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35
[0]	validation_0-rmse:0.31534
[500]	validation_0-rmse:0.28576
[1000]	validation_0-rmse:0.28320
[1500]	validation_0-rmse:0.28200
[2000]	validation_0-rmse:0.28157
[2500]	validation_0-rmse:0.28144
[2975]	validation_0-rmse:0.28148

Fold 2
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35
[0]	validation_0-rmse:0.31314
[500]	validation_0-rmse:0.28410
[1000]	validation_0-rmse:0.28170
[1500]	validation_0-rmse:0.28071
[2000]	validation_0-rmse:0.28044
[2500]	validation_0-rmse:0.28038
[2890]	validation_0-rmse:0.28048

Fold 3
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35
[0]	validation_0-rmse:0.31538
[500]	validation_0-rmse:0.28344
[1000]	validation_0-rmse:0.28027
[1500]	validation_0-rmse:0.27916
[2000]	validation_0-rmse:0.27873
[2500]	validation_0-rmse:0.27869
[2999]	validation_0-rmse:0.27876



In [17]:
if VALIDATE:
    lgb_km = CrossVal(model='lgb')
    lgb_km_oof_pred, lgb_km_test_pred = lgb_km.train_survival_model(
        train=train,
        test=test,
        cat_feats=cat_feats,
        covariate=covariate,
        target_col='mean_km_dras',
        params=lgb_params_km
    )


Fold 1
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 2
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 3
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 4
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 5
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35


In [18]:
if VALIDATE:
    lgb_na = CrossVal(model='lgb')
    lgb_na_oof_pred, lgb_na_test_pred = lgb_na.train_survival_model(
        train=train,
        test=test,
        cat_feats=cat_feats,
        covariate=covariate,
        target_col='na',
        params=lgb_params_na
    )


Fold 1
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 2
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 3
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 4
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 5
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35


In [19]:
if VALIDATE:
    lgb2_km = CrossVal(model='lgb2')
    lgb2_km_oof_pred, lgb2_km_test_pred = lgb2_km.train_survival_model(
        train=train,
        test=test,
        cat_feats=cat_feats,
        covariate=covariate,
        target_col='mean_km_dras',
        params=lgb2_params_km
    )


Fold 1
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 2
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 3
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 4
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 5
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35


In [20]:
if VALIDATE:
    lgb2_na = CrossVal(model='lgb2')
    lgb2_na_oof_pred, lgb2_na_test_pred = lgb2_na.train_survival_model(
        train=train,
        test=test,
        cat_feats=cat_feats,
        covariate=covariate,
        target_col='na',
        params=lgb2_params_na
    )


Fold 1
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 2
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 3
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 4
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35

Fold 5
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35


In [21]:
if VALIDATE:
    cat_km = CrossVal(model='cat')
    cat_km_oof_pred, cat_km_test_pred = cat_km.train_survival_model(
        train=train,
        test=test,
        cat_feats=cat_feats,
        covariate=covariate,
        target_col='mean_km_dras',
        params=cat_params_km
    )


Fold 1
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35
0:	learn: 0.2730150	test: 0.2726025	best: 0.2726025 (0)	total: 64.2ms	remaining: 5m 20s
500:	learn: 0.2343220	test: 0.2459057	best: 0.2459057 (500)	total: 3.79s	remaining: 34s
1000:	learn: 0.2256904	test: 0.2445280	best: 0.2445280 (1000)	total: 7.48s	remaining: 29.9s
1500:	learn: 0.2192724	test: 0.2440803	best: 0.2440702 (1476)	total: 11.3s	remaining: 26.3s
2000:	learn: 0.2137613	test: 0.2439581	best: 0.2439327 (1971)	total: 15s	remaining: 22.5s
2500:	learn: 0.2090039	test: 0.2440256	best: 0.2438846 (2098)	total: 18.6s	remaining: 18.6s
3000:	learn: 0.2047188	test: 0.2441403	best: 0.2438846 (2098)	total: 22.3s	remaining: 14.9s
3500:	learn: 0.2008141	test: 0.2442752	best: 0.2438846 (2098)	total: 26.7s	remaining: 11.5s
4000:	learn: 0.1973202	test: 0.2444625	best: 0.2438846 (2098)	total: 30.8s	remaining: 7.69s
4500:	learn: 0.1941600	test: 0.2448842	best: 0.2438846 (2098)	total: 34.7s	r

In [22]:
if VALIDATE:
    cat_na = CrossVal(model='cat')
    cat_na_oof_pred, cat_na_test_pred = cat_na.train_survival_model(
        train=train,
        test=test,
        cat_feats=cat_feats,
        covariate=covariate,
        target_col='na',
        params=cat_params_na
    )


Fold 1
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35
0:	learn: 0.3145132	test: 0.3151955	best: 0.3151955 (0)	total: 42ms	remaining: 1m 38s
500:	learn: 0.2654000	test: 0.2841502	best: 0.2841502 (500)	total: 21.2s	remaining: 1m 17s
1000:	learn: 0.2471218	test: 0.2817564	best: 0.2817564 (1000)	total: 41.3s	remaining: 55.3s
1500:	learn: 0.2310242	test: 0.2810928	best: 0.2810836 (1494)	total: 1m	remaining: 33.9s
2000:	learn: 0.2169750	test: 0.2810255	best: 0.2809280 (1895)	total: 1m 20s	remaining: 13.7s
2340:	learn: 0.2080913	test: 0.2810551	best: 0.2809280 (1895)	total: 1m 34s	remaining: 0us

bestTest = 0.2809279567
bestIteration = 1895

Shrink model to first 1896 iterations.

Fold 2
Number of features: 97
Non-categorical features: 27
Categorical features: 35
Encoded features: 35
0:	learn: 0.3150577	test: 0.3129817	best: 0.3129817 (0)	total: 40.8ms	remaining: 1m 35s
500:	learn: 0.2651664	test: 0.2826787	best: 0.2826787 (500)	total: 20.7s

In [23]:
# Caruana's weighted ensembling method
def caruana_ensemble_selection(base_preds, y_true, metric_func, max_models=None):
    """
    Implement Caruana's ensemble selection algorithm
    
    Args:
        base_preds: List of base model predictions
        y_true: True target values 
        metric_func: Function to evaluate ensemble performance
        max_models: Maximum number of models to include (can include same model multiple times)
    
    Returns:
        ensemble_preds: The final ensemble predictions
        model_weights: Dictionary of relative frequency for each model
    """
    if max_models is None:
        max_models = len(base_preds) * 10
        
    model_names = [f'model_{i}' for i in range(len(base_preds))]
    ensemble_preds = np.zeros_like(y_true, dtype=float)
    model_counts = {name: 0 for name in model_names}
    
    # Greedy forward selection
    for _ in range(max_models):
        best_score = float('-inf')
        best_model = None
        
        # Try adding each model to the current ensemble
        for i, model_preds in enumerate(base_preds):
            # Create temporary ensemble with this model added
            temp_ensemble = (ensemble_preds * sum(model_counts.values()) + model_preds) / (sum(model_counts.values()) + 1) if sum(model_counts.values()) > 0 else model_preds
            score = metric_func(y_true, temp_ensemble)
            
            if score > best_score:
                best_score = score
                best_model = i
        
        # If no improvement, break
        if best_model is None:
            break
            
        # Update ensemble with best model
        if sum(model_counts.values()) > 0:
            ensemble_preds = (ensemble_preds * sum(model_counts.values()) + base_preds[best_model]) / (sum(model_counts.values()) + 1)
        else:
            ensemble_preds = base_preds[best_model].copy()
            
        model_counts[model_names[best_model]] += 1
    
    # Convert counts to weights
    total_models = sum(model_counts.values())
    weights = {name: count/total_models for name, count in model_counts.items()}
    
    return ensemble_preds, weights

In [24]:
VALIDATE = False
if VALIDATE:
    base_model_preds = [
        #xgb_km_test_pred, 
        #xgb_na_test_pred, 
        #lgb_km_test_pred, 
        lgb_na_test_pred, 
        #lgb2_km_test_pred, 
        lgb2_na_test_pred, 
        #cat_km_test_pred,
        cat_na_test_pred
    ]
    
    oof_preds = [
        #xgb_km_oof_pred, 
        xgb_na_oof_pred, 
        #lgb_km_oof_pred, 
        #lgb_na_oof_pred, 
        #lgb2_km_oof_pred, 
        lgb2_na_oof_pred, 
        #cat_km_oof_pred,
        cat_na_oof_pred
    ]
    
    # This should return higher values for better predictions
    def custom_score(y_true, y_pred):
        y_pred_df = train[["ID"]].copy()
        y_pred_df["prediction"] = y_pred
        m = score(train[["ID", "efs", "efs_time", "race_group"]].copy(), y_pred_df, "ID")
        print(m)
        return m
    
    # Apply Caruana ensemble selection on validation data
    _, model_weights = caruana_ensemble_selection(
        oof_preds, 
        train["mean_km_dras"],
        custom_score,
        max_models=20
    )
    
    print("Model weights from Caruana ensemble selection:")
    for i, (model_name, weight) in enumerate(model_weights.items()):
        print(f"{['xgb_na', 'lgb_na', 'lgb2_na', 'cat_na'][i]}: {weight:.4f}")
    
    # Apply these weights to test predictions
    weighted_test_pred = np.zeros_like(xgb_km_test_pred)
    for i, weight_key in enumerate(model_weights.keys()):
        if model_weights[weight_key] > 0:
            weighted_test_pred += base_model_preds[i] * model_weights[weight_key]
    
    # Create submission
    submission = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv')
    submission['prediction'] = weighted_test_pred
    submission.to_csv('submission.csv', index=False)
    display(submission.head())

In [25]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import make_scorer

def stacking_with_cross_validation(
    base_model_preds, 
    y_true, 
    cv_folds=5, 
    meta_model='lasso', 
    custom_scorer=None,
    id_column=None,
    additional_columns=None
):
    """
    Implements stacking with proper cross-validation to find optimal model weights
    
    Args:
        base_model_preds: List of base model predictions [model_1_preds, model_2_preds, ...]
        y_true: True target values (pandas Series or numpy array)
        cv_folds: Number of cross-validation folds
        meta_model: Type of meta-model ('lasso', 'ridge', or 'elasticnet')
        custom_scorer: Custom scoring function (should be compatible with scikit-learn's make_scorer)
        id_column: Optional ID column for custom scorers that require it
        additional_columns: Optional additional columns for custom scorers
        
    Returns:
        weights: Coefficients for each base model
        meta_model: Trained meta-model
        stacked_preds: Final ensemble predictions using meta-model
    """
    n_models = len(base_model_preds)
    n_samples = len(y_true)
    
    # Convert predictions to numpy arrays 
    base_model_preds = [np.array(preds) for preds in base_model_preds]
    
    X = np.column_stack(base_model_preds)
    
    if custom_scorer and (id_column is not None or additional_columns is not None):
        data_for_scoring = pd.DataFrame()
        if id_column is not None:
            data_for_scoring["ID"] = id_column
        if additional_columns is not None:
            for col_name, col_data in additional_columns.items():
                data_for_scoring[col_name] = col_data
        
        # Create a scorer that works with our custom function
        def scorer_wrapper(y_true, y_pred):
            # Create a prediction DataFrame
            y_pred_df = pd.DataFrame({"ID": data_for_scoring["ID"], "prediction": y_pred})
            return custom_scorer(data_for_scoring.copy(), y_pred_df, "ID")
            
        sklearn_scorer = make_scorer(scorer_wrapper, greater_is_better=True)
    else:
        sklearn_scorer = None
    
    # Setup cross-validation
    kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    # Select the meta-model
    if meta_model == 'lasso':
        model = LassoCV(
            cv=cv_folds, 
            fit_intercept=False, 
            positive=True,
            selection='random',
            max_iter=10000,
            alphas=[0.001, 0.01, 0.1, 1.0, 10.0]
        )
    elif meta_model == 'ridge':
        model = RidgeCV(
            cv=cv_folds,
            fit_intercept=False,
            alphas=[0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
            scoring=sklearn_scorer
        )
    elif meta_model == 'elasticnet':
        model = ElasticNetCV(
            cv=cv_folds,
            fit_intercept=False,
            positive=True,
            max_iter=10000,
            l1_ratio=[.1, .5, .7, .9, .95, .99, 1],
            alphas=[0.001, 0.01, 0.1, 1.0, 10.0]
        )
    else:
        raise ValueError("meta_model must be 'lasso', 'ridge', or 'elasticnet'")
    
    # Fit the meta-model
    model.fit(X, y_true)
    weights = model.coef_
    
    # Normalize weights to sum to 1 
    if np.sum(weights) > 0:
        weights = weights / np.sum(weights)
    
    # Calculate the stacked predictions
    stacked_preds = model.predict(X)
    
    # Return the model weights and predictions
    model_weights = {f'model_{i}': w for i, w in enumerate(weights)}
    
    return model_weights, model, stacked_preds

In [26]:
VALIDATE = True
if VALIDATE:
    # Prepare list of all model predictions
    base_model_test_preds = [
        #xgb_km_test_pred, 
        xgb_na_test_pred, 
        #lgb_km_test_pred, 
        #lgb_na_test_pred, 
        #lgb2_km_test_pred, 
        lgb2_na_test_pred, 
        #cat_km_test_pred,
        cat_na_test_pred
    ]
    
    # For validation, we need ground truth values
    # Using OOF predictions to determine weights
    base_model_oof_preds = [
        #xgb_km_oof_pred, 
        xgb_na_oof_pred, 
        #lgb_km_oof_pred, 
        #lgb_na_oof_pred, 
        #lgb2_km_oof_pred, 
        lgb2_na_oof_pred, 
        #cat_km_oof_pred,
        cat_na_oof_pred
    ]
    
    # Define mapping from model index to name
    model_names = [#'xgb_km', 
                   'xgb_na', 
                   #'lgb_km', 
                   #'lgb_na', 
                   #'lgb2_km', 
                   'lgb2_na', 
                   #'cat_km', 
                   'cat_na']
    
    # Define your custom scoring function
    def custom_score(y_true_df, y_pred_df, id_col):
        # Replace with your actual scoring function
        return score(y_true_df, y_pred_df, id_col)
    
    # Prepare additional columns needed for scoring
    additional_cols = {
        "efs": train["efs"],
        "efs_time": train["efs_time"],
        "race_group": train["race_group"]
    }
    
    # Apply stacking with cross-validation
    print("Training stacking ensemble with cross-validation...")
    model_weights, meta_model, _ = stacking_with_cross_validation(
        base_model_oof_preds,
        train["na"],
        cv_folds=10,
        meta_model='lasso', 
        custom_scorer=custom_score,
        id_column=train["ID"],
        additional_columns=additional_cols
    )
    
    # Print model weights
    print("\nModel weights from Stacking with Cross-Validation:")
    for i, (model_key, weight) in enumerate(model_weights.items()):
        print(f"{model_names[i]}: {weight:.4f}")
    
    # Apply meta-model to test predictions
    test_X = np.column_stack(base_model_test_preds)
    stacked_test_pred = meta_model.predict(test_X)
    
    # 1. Stacking
    stacked_oof_pred = meta_model.predict(np.column_stack(base_model_oof_preds))
    y_true_val = train[["ID", "efs", "efs_time", "race_group"]].copy()
    
    y_pred_stacked = train[["ID"]].copy()
    y_pred_stacked["prediction"] = stacked_oof_pred
    stacked_val_score = score(y_true_val.copy(), y_pred_stacked.copy(), "ID")
    
    # 2. Original rank-based ensemble
    ranked_test_pred = sum(rankdata(pred) for pred in base_model_test_preds)
    ranked_oof_pred = sum(rankdata(pred) for pred in base_model_oof_preds)
    y_pred_ranked = train[["ID"]].copy()
    y_pred_ranked["prediction"] = ranked_oof_pred
    ranked_val_score = score(y_true_val.copy(), y_pred_ranked.copy(), "ID")
    
    # 3. Simple average (as a baseline)
    avg_test_pred = sum(pred for pred in base_model_test_preds) / len(base_model_test_preds)
    avg_oof_pred = sum(pred for pred in base_model_oof_preds) / len(base_model_oof_preds)
    y_pred_avg = train[["ID"]].copy()
    y_pred_avg["prediction"] = avg_oof_pred
    avg_val_score = score(y_true_val.copy(), y_pred_avg.copy(), "ID")
    
    print("\nValidation Performance Comparison:")
    print(f"Stacking with CV: {stacked_val_score}")
    print(f"Rank-based ensemble: {ranked_val_score}")
    print(f"Simple average: {avg_val_score}")
    
    # Create a final evaluation of which method performed best
    methods = ["Stacking", "Rank-based", "Simple Average"]
    scores = [stacked_val_score, ranked_val_score, avg_val_score]
    best_method = methods[np.argmax(scores)]
    
    print(f"\nBased on validation performance, the {best_method} method performs best.")
    print("Consider submitting this as your final prediction.")

    submission = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv')
    submission['prediction'] = stacked_test_pred
    submission.to_csv('submission.csv', index=False)
    display(submission.head())

Training stacking ensemble with cross-validation...

Model weights from Stacking with Cross-Validation:
xgb_na: 0.5595
lgb2_na: 0.1162
cat_na: 0.3243

Validation Performance Comparison:
Stacking with CV: 0.6787770242052902
Rank-based ensemble: 0.6786545969004412
Simple average: 0.6786524282790635

Based on validation performance, the Stacking method performs best.
Consider submitting this as your final prediction.


,ID,prediction
0,28800,-0.834369
1,28801,-0.493271
2,28802,-0.919766
